In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", context='poster')
import math

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import sys
sys.path.append('..')
from model_equations_separate_NC_store_numba import *


In [4]:
import seaborn_figure_settings

In [5]:
morder = ['MIXOTROPH', 'OVERFLOW', 'EXOENZYME',  'ROS',]
media_order = ['lowN', 'pro99']
mpalette = 'twilight'
mpalette = sns.color_palette(mpalette, n_colors=4)
#morder = [morder[1]] + morder[3:]
#mpalette = [mpalette[1]] + mpalette[3:]
sns.color_palette(mpalette)

[(0.42790652284925834, 0.5649543060909209, 0.7513944352191484),
 (0.3269137124539796, 0.1157873496841953, 0.4865864649569746),
 (0.39199887556812907, 0.09691583650296684, 0.2959521200550908),
 (0.7531856636904657, 0.45841199172949604, 0.3686223664223477)]

In [6]:
gorder = [ 'Strong', 'Sustained', 'Inhibited', 'Weak', 'Axenic',   'Other']
gpalette = sns.color_palette('Set2', n_colors=4).as_hex() + ['black', 'lightgrey']
gpalette = ['#882255', '#CC6677', '#332288', 
             '#44AA99','#88CCEE',
            '#D0CFCA',  ]

sns.color_palette(gpalette)

[(0.5333333333333333, 0.13333333333333333, 0.3333333333333333),
 (0.8, 0.4, 0.4666666666666667),
 (0.2, 0.13333333333333333, 0.5333333333333333),
 (0.26666666666666666, 0.6666666666666666, 0.6),
 (0.5333333333333333, 0.8, 0.9333333333333333),
 (0.8156862745098039, 0.8117647058823529, 0.792156862745098)]

In [7]:
dpath = '/fast_data/Osnat/RECYCLE_MODEL/results/shgo2/out'
dpath = '../results/final/ponly/'

In [8]:
import glob
import os
glob.glob(os.path.join(dpath, "*_sum*"))

['../results/final/ponly/ponly_monte_add_ROS_round2_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_EXOENZYME-MIXOTROPH_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_OVERFLOW-ROS_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_add_OVERFLOW_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_OVERFLOW-MIXOTROPH_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_fix_OVERFLOW_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_EXOENZYME-ROS_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_OVERFLOW-EXOENZYME-MIXOTROPH_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_OVERFLOW-ROS-MIXOTROPH_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_EXOENZYME-ROS-MIXOTROPH_clean_sum.csv.gz',
 '../results/final/ponly/monte_extend_ponly_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_ROS-MIXOTROPH-OVERFLOW-EXOENZYME_clean_sum.csv.gz',
 '../results/final/ponly/monte_ponly_ROS-MIXOTROP

In [43]:
sim_df1 = pd.read_csv(os.path.join(dpath,'monte_ponly_clean_sum.csv.gz',))
sim_df3 = pd.read_csv(os.path.join(dpath,'monte_ponly_fix_OVERFLOW_clean_sum.csv.gz',))


In [44]:
# remove buggy overflow simulations
sim_df1 = sim_df1.loc[~sim_df1.run_id.str.startswith('monte_ponly_OVERFLOW')].copy()


In [45]:
sim_df1[['id', 'media', 'which', 'model', 'hash']] =     sim_df1.run_id.str.rsplit('_', n=4, expand=True)
sim_df3[['id', 'media', 'which', 'model', 'hash']] =     sim_df3.run_id.str.rsplit('_', n=4, expand=True)


In [46]:
sim_df1['VPRO_nocluster'] = sim_df1.idx.str.replace('monte_ponly_','vpro_', regex=False)
sim_df1['VPRO_nocluster'] = sim_df1['VPRO_nocluster'].str.replace('_monte__','_', regex=False)
for m in morder:
    sim_df1['VPRO_nocluster'] = sim_df1['VPRO_nocluster'].str.replace(f'_{m}_','_', regex=False)


In [47]:
sim_df3['VPRO'] = sim_df3.idx.str.replace('monte_ponly_fix_OVERFLOW_','vpro_2_', regex=False)
sim_df3['VPRO'] = sim_df3['VPRO'].str.replace('_monte__','_', regex=False)
for m in morder:
    sim_df3['VPRO'] = sim_df3['VPRO'].str.replace(f'_{m}_','_', regex=False)


In [48]:
sim_df3['VPRO_nocluster'] = sim_df3['VPRO']

In [49]:
df = pd.concat([
    sim_df1,
    sim_df3,    
], ignore_index=True)

In [50]:
df

,VmaxICp,VmaxICh,VmaxINp,VmaxINh,VmaxOCp,VmaxOCh,VmaxONp,VmaxONh,KICp,KICh,...,ROSmaxD,run_id,idx,id,media,which,model,hash,VPRO_nocluster,VPRO
0,0.000002,0.0,0.000003,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,37.000000,1.0,...,0.0,monte_ponly_EXOENZYME_76_monte__69_pro99_ponly...,monte_ponly_EXOENZYME_76_monte__69_EXOENZYME,monte_ponly_EXOENZYME_76_monte__69,pro99,ponly,EXOENZYME,h2291012746155587316,vpro_76_69_EXOENZYME,NaN
1,0.000006,0.0,0.000035,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,24.605317,1.0,...,0.0,monte_ponly_EXOENZYME_86_monte__38_lowN_ponly_...,monte_ponly_EXOENZYME_86_monte__38_EXOENZYME,monte_ponly_EXOENZYME_86_monte__38,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_86_38_EXOENZYME,NaN
2,0.000030,0.0,0.000010,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,215.112381,1.0,...,0.0,monte_ponly_EXOENZYME_29_monte__60_pro99_ponly...,monte_ponly_EXOENZYME_29_monte__60_EXOENZYME,monte_ponly_EXOENZYME_29_monte__60,pro99,ponly,EXOENZYME,h2291012746155587316,vpro_29_60_EXOENZYME,NaN
3,0.000005,0.0,0.000006,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,8.853069,1.0,...,0.0,monte_ponly_EXOENZYME_23_monte__20_lowN_ponly_...,monte_ponly_EXOENZYME_23_monte__20_EXOENZYME,monte_ponly_EXOENZYME_23_monte__20,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_23_20_EXOENZYME,NaN
4,0.000004,0.0,0.000007,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,7.193827,1.0,...,0.0,monte_ponly_EXOENZYME_20_monte__81_lowN_ponly_...,monte_ponly_EXOENZYME_20_monte__81_EXOENZYME,monte_ponly_EXOENZYME_20_monte__81,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_20_81_EXOENZYME,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99649,0.000003,0.0,0.000006,0.000067,0.0,6.666667e-05,0.0,6.666667e-05,3.840946,1.0,...,0.0,monte_ponly_fix_OVERFLOW_36_monte__18_lowN_pon...,monte_ponly_fix_OVERFLOW_36_monte__18_OVERFLOW,monte_ponly_fix_OVERFLOW_36_monte__18,lowN,ponly,OVERFLOW,h6878164821315599434,vpro_2_36_18_OVERFLOW,vpro_2_36_18_OVERFLOW
99650,0.000006,0.0,0.000028,0.000067,0.0,6.666667e-05,0.0,6.666667e-05,213.432130,1.0,...,0.0,monte_ponly_fix_OVERFLOW_75_monte__75_pro99_po...,monte_ponly_fix_OVERFLOW_75_monte__75_OVERFLOW,monte_ponly_fix_OVERFLOW_75_monte__75,pro99,ponly,OVERFLOW,h6878164821315599434,vpro_2_75_75_OVERFLOW,vpro_2_75_75_OVERFLOW
99651,0.000007,0.0,0.000006,0.000067,0.0,6.666667e-05,0.0,6.666667e-05,166.221407,1.0,...,0.0,monte_ponly_fix_OVERFLOW_37_monte__69_lowN_pon...,monte_ponly_fix_OVERFLOW_37_monte__69_OVERFLOW,monte_ponly_fix_OVERFLOW_37_monte__69,lowN,ponly,OVERFLOW,h6878164821315599434,vpro_2_37_69_OVERFLOW,vpro_2_37_69_OVERFLOW
99652,0.000016,0.0,0.000006,0.000067,0.0,6.666667e-05,0.0,6.666667e-05,6.774866,1.0,...,0.0,monte_ponly_fix_OVERFLOW_22_monte__89_lowN_pon...,monte_ponly_fix_OVERFLOW_22_monte__89_OVERFLOW,monte_ponly_fix_OVERFLOW_22_monte__89,lowN,ponly,OVERFLOW,h6878164821315599434,vpro_2_22_89_OVERFLOW,vpro_2_22_89_OVERFLOW


In [51]:
df = df.loc[df.media.isin(['lowN'])].copy()

In [52]:
df[['model']].value_counts()

model    
MIXOTROPH    9997
EXOENZYME    9959
MIN          9958
ROS          9958
OVERFLOW     9955
Name: count, dtype: int64

In [53]:
vpro_df = pd.read_csv('../ML_model/versatile_vpros.csv')

In [54]:
vpro_df['VPRO_nocluster'] =  vpro_df.VPRO.str.replace(r'_C\d$', '', regex=True)

In [55]:
vpro_df

,model,Phase,VPRO,Inhibited,Neutral,Other,Strong,Sustained,Weak,pos_interaction,...,Versatile_STR,MODEL,EXOENZYME,OVERFLOW,ROS,MIXOTROPH,MIN,% positive,Versatile_STR_,VPRO_nocluster
0,EXOENZYME,add,vpro_11_62_EXOENZYME_C1_15,0,0,0.0,0.0,0.0,24,0,...,Non-Versatile VPRO,E,True,False,False,False,False,0.000000,Non-Versatile VPRO,vpro_11_62_EXOENZYME_C1_15
1,EXOENZYME,add,vpro_11_62_EXOENZYME_C1_20,0,1,2.0,0.0,9.0,77,9,...,Versatile VPRO,E,True,False,False,False,False,0.101124,Versatile VPRO,vpro_11_62_EXOENZYME_C1_20
2,EXOENZYME,add,vpro_11_62_EXOENZYME_C1_22,0,0,0.0,0.0,0.0,27,0,...,Non-Versatile VPRO,E,True,False,False,False,False,0.000000,Non-Versatile VPRO,vpro_11_62_EXOENZYME_C1_22
3,EXOENZYME,add,vpro_11_62_EXOENZYME_C1_32,0,3,0.0,0.0,0.0,23,0,...,Non-Versatile VPRO,E,True,False,False,False,False,0.000000,Non-Versatile VPRO,vpro_11_62_EXOENZYME_C1_32
4,EXOENZYME,add,vpro_11_62_EXOENZYME_C1_37,0,29,0.0,0.0,0.0,35,0,...,Non-Versatile VPRO,E,True,False,False,False,False,0.000000,Non-Versatile VPRO,vpro_11_62_EXOENZYME_C1_37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19771,ROS-MIXOTROPH-OVERFLOW-EXOENZYME,init,vpro_82_9_ROS-MIXOTROPH-OVERFLOW-EXOENZYME,0,0,0.0,0.0,0.0,209,0,...,Non-Versatile VPRO,EORM,True,True,True,True,False,0.000000,Non-Versatile VPRO,vpro_82_9_ROS-MIXOTROPH-OVERFLOW-EXOENZYME
19772,ROS-MIXOTROPH-OVERFLOW-EXOENZYME,init,vpro_86_79_ROS-MIXOTROPH-OVERFLOW-EXOENZYME,0,19,4.0,0.0,0.0,237,0,...,Non-Versatile VPRO,EORM,True,True,True,True,False,0.000000,Non-Versatile VPRO,vpro_86_79_ROS-MIXOTROPH-OVERFLOW-EXOENZYME
19773,ROS-MIXOTROPH-OVERFLOW-EXOENZYME,init,vpro_95_6_ROS-MIXOTROPH-OVERFLOW-EXOENZYME,0,16,56.0,236.0,1.0,0,237,...,Non-Versatile VPRO,EORM,True,True,True,True,False,0.766990,Non-Versatile VPRO,vpro_95_6_ROS-MIXOTROPH-OVERFLOW-EXOENZYME
19774,ROS-MIXOTROPH-OVERFLOW-EXOENZYME,init,vpro_97_54_ROS-MIXOTROPH-OVERFLOW-EXOENZYME,0,127,123.0,0.0,17.0,15,17,...,Versatile VPRO,EORM,True,True,True,True,False,0.060284,Versatile VPRO,vpro_97_54_ROS-MIXOTROPH-OVERFLOW-EXOENZYME


In [56]:
sim_df3.VPRO_nocluster.unique()

array(['vpro_2_33_91_OVERFLOW', 'vpro_2_85_7_OVERFLOW',
       'vpro_2_5_8_OVERFLOW', ..., 'vpro_2_53_7_OVERFLOW',
       'vpro_2_36_18_OVERFLOW', 'vpro_2_68_22_OVERFLOW'], dtype=object)

In [57]:
vpro_df.loc[
    #~vpro_df.VPRO.isin(df.VPRO) 
    ~vpro_df.VPRO_nocluster.isin(df.VPRO_nocluster)
][['Phase', 'model']].value_counts()


Phase   model                           
extend  OVERFLOW-MIXOTROPH                  1618
        EXOENZYME-ROS-MIXOTROPH             1437
        OVERFLOW-EXOENZYME-MIXOTROPH        1400
        OVERFLOW-ROS-MIXOTROPH              1335
        ROS-MIXOTROPH-OVERFLOW-EXOENZYME    1237
round2  ROS                                 1222
extend  MIXOTROPH                           1204
        OVERFLOW-EXOENZYME                  1084
        OVERFLOW-ROS                        1033
        EXOENZYME-MIXOTROPH                  990
        OVERFLOW-ROS-EXOENZYME               987
        OVERFLOW                             915
        ROS-MIXOTROPH                        876
        EXOENZYME-ROS                        789
        ROS                                  698
        MIN                                  673
        EXOENZYME                            598
add     OVERFLOW                             559
        EXOENZYME                            362
        ROS                 

In [58]:
df.columns

Index(['VmaxICp', 'VmaxICh', 'VmaxINp', 'VmaxINh', 'VmaxOCp', 'VmaxOCh',
       'VmaxONp', 'VmaxONh', 'KICp', 'KICh', 'KINp', 'KINh', 'KOCp', 'KOCh',
       'KONp', 'KONh', 'Mp', 'Mh', 'gammaDp', 'gammaDh', 'Rp', 'Rh', 'QCmaxp',
       'QCminp', 'QCmaxh', 'QCminh', 'Kmtbp', 'Kmtbh', 'bp', 'bh', 'r0p',
       'r0h', 'KdecayDON', 'KprodEXOp', 'KprodEXOh', 'OverflowMode',
       'Koverflowp', 'Koverflowh', 'ROSMode', 'KprodROSp', 'KprodROSh',
       'KlossROSp', 'KlossROSh', 'KdecayROS', 'omegaP', 'omegaH', 'ROSmaxD',
       'run_id', 'idx', 'id', 'media', 'which', 'model', 'hash',
       'VPRO_nocluster', 'VPRO'],
      dtype='object')

In [59]:
df = pd.merge(df, vpro_df[['VPRO_nocluster', 'Versatile_STR']], on='VPRO_nocluster', how='left')



In [60]:
df = df.loc[~df.Versatile_STR.isna()]


In [61]:
#df['Versatile_STR'] = df.Versatile_STR.fillna('Not VPRO')

In [62]:
df['Versatile_STR'].value_counts()


Versatile_STR
Non-Versatile VPRO    118
Versatile VPRO         45
Name: count, dtype: int64

In [64]:
df.head()

,VmaxICp,VmaxICh,VmaxINp,VmaxINh,VmaxOCp,VmaxOCh,VmaxONp,VmaxONh,KICp,KICh,...,run_id,idx,id,media,which,model,hash,VPRO_nocluster,VPRO,Versatile_STR
1407,0.000038,0.0,0.000028,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,37.000000,1.0,...,monte_ponly_EXOENZYME_58_monte__74_lowN_ponly_...,monte_ponly_EXOENZYME_58_monte__74_EXOENZYME,monte_ponly_EXOENZYME_58_monte__74,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_58_74_EXOENZYME,NaN,Versatile VPRO
1604,0.000020,0.0,0.000023,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,37.000000,1.0,...,monte_ponly_EXOENZYME_96_monte__67_lowN_ponly_...,monte_ponly_EXOENZYME_96_monte__67_EXOENZYME,monte_ponly_EXOENZYME_96_monte__67,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_96_67_EXOENZYME,NaN,Non-Versatile VPRO
1666,0.000028,0.0,0.000030,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,10.672295,1.0,...,monte_ponly_EXOENZYME_57_monte__7_lowN_ponly_E...,monte_ponly_EXOENZYME_57_monte__7_EXOENZYME,monte_ponly_EXOENZYME_57_monte__7,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_57_7_EXOENZYME,NaN,Versatile VPRO
1871,0.000029,0.0,0.000017,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,129.526524,1.0,...,monte_ponly_EXOENZYME_49_monte__10_lowN_ponly_...,monte_ponly_EXOENZYME_49_monte__10_EXOENZYME,monte_ponly_EXOENZYME_49_monte__10,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_49_10_EXOENZYME,NaN,Non-Versatile VPRO
1925,0.000055,0.0,0.000032,0.000002,0.0,5.541667e-08,0.0,4.277778e-08,68.961589,1.0,...,monte_ponly_EXOENZYME_11_monte__62_lowN_ponly_...,monte_ponly_EXOENZYME_11_monte__62_EXOENZYME,monte_ponly_EXOENZYME_11_monte__62,lowN,ponly,EXOENZYME,h2291012746155587316,vpro_11_62_EXOENZYME,NaN,Versatile VPRO


In [65]:
df.VPRO.isna().sum()

108

In [66]:
df = df.reset_index(drop=True)

In [67]:
df.groupby(['model',]).size() #.value_counts()

model
EXOENZYME    43
MIXOTROPH    35
OVERFLOW     55
ROS          30
dtype: int64

In [69]:
df.columns

Index(['VmaxICp', 'VmaxICh', 'VmaxINp', 'VmaxINh', 'VmaxOCp', 'VmaxOCh',
       'VmaxONp', 'VmaxONh', 'KICp', 'KICh', 'KINp', 'KINh', 'KOCp', 'KOCh',
       'KONp', 'KONh', 'Mp', 'Mh', 'gammaDp', 'gammaDh', 'Rp', 'Rh', 'QCmaxp',
       'QCminp', 'QCmaxh', 'QCminh', 'Kmtbp', 'Kmtbh', 'bp', 'bh', 'r0p',
       'r0h', 'KdecayDON', 'KprodEXOp', 'KprodEXOh', 'OverflowMode',
       'Koverflowp', 'Koverflowh', 'ROSMode', 'KprodROSp', 'KprodROSh',
       'KlossROSp', 'KlossROSh', 'KdecayROS', 'omegaP', 'omegaH', 'ROSmaxD',
       'run_id', 'idx', 'id', 'media', 'which', 'model', 'hash',
       'VPRO_nocluster', 'VPRO', 'Versatile_STR'],
      dtype='object')

In [71]:
df = df.drop(columns=['idx', 'id', 'media', 'which', 'model', 'hash',
       'VPRO_nocluster', 'VPRO', 'Versatile_STR'])

In [75]:
df.to_csv(os.path.join(dpath,'vpros_init_sum.csv'), index=False)

In [72]:
dpath

'../results/final/ponly/'